## Cleanup Process

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from copy import deepcopy

importar e dropar as colunas inuteis para treinar

In [3]:
df = pd.read_csv('metadata_3d.csv')
df.head()

,patient_id,nodule_idx,malignancy,is_cancer,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,LIDC-IDRI-0001,0,5.0,True,0.972773,0.858091,19.588162,22.827597,31.038346,32.075141,...,450.048684,0.003344,6.525033,0.363301,481.250120,0.466158,0.001206,3162.569676,0.328881,1.274217
1,LIDC-IDRI-0002,0,4.5,True,0.795595,0.747403,21.632902,28.944112,36.659875,30.945300,...,191.254121,0.007831,6.967980,0.141313,2172.933577,2.040966,0.001017,1057.086620,0.053434,0.926980
2,LIDC-IDRI-0003,0,2.0,False,0.768615,0.549141,14.561133,26.516200,26.873199,31.855156,...,175.560472,0.013967,6.693464,0.282061,77.874166,0.540909,0.003731,1918.619966,0.090857,4.007826
3,LIDC-IDRI-0003,1,4.5,True,0.836843,0.690356,16.647824,24.114835,26.959597,30.922607,...,285.645229,0.006369,6.398453,0.405739,208.855594,0.375968,0.002146,2521.046243,0.342239,1.578325
4,LIDC-IDRI-0003,2,3.5,Ambiguous,0.667540,0.638922,7.316637,11.451529,11.145619,12.548361,...,202.401011,0.031783,5.650718,0.662835,2.307728,0.176433,0.012777,3284.429889,1.238819,5.874138


In [4]:
df.drop("patient_id", axis=1, inplace=True)
df.drop("nodule_idx", axis=1, inplace=True)
#df.head()

In [5]:
df = df.loc[:, df.nunique() > 1]
df = df.loc[:,~df.T.duplicated()] # df.T e a transposta, depois trata as colunas como linhas para encontrar repetidas
df.head()

,malignancy,is_cancer,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,5.0,True,0.972773,0.858091,19.588162,22.827597,31.038346,32.075141,26.977005,34.406945,...,450.048684,0.003344,6.525033,0.363301,481.250120,0.466158,0.001206,3162.569676,0.328881,1.274217
1,4.5,True,0.795595,0.747403,21.632902,28.944112,36.659875,30.945300,37.724947,40.384819,...,191.254121,0.007831,6.967980,0.141313,2172.933577,2.040966,0.001017,1057.086620,0.053434,0.926980
2,2.0,False,0.768615,0.549141,14.561133,26.516200,26.873199,31.855156,25.552264,32.630357,...,175.560472,0.013967,6.693464,0.282061,77.874166,0.540909,0.003731,1918.619966,0.090857,4.007826
3,4.5,True,0.836843,0.690356,16.647824,24.114835,26.959597,30.922607,24.750701,30.922607,...,285.645229,0.006369,6.398453,0.405739,208.855594,0.375968,0.002146,2521.046243,0.342239,1.578325
4,3.5,Ambiguous,0.667540,0.638922,7.316637,11.451529,11.145619,12.548361,12.934109,13.267970,...,202.401011,0.031783,5.650718,0.662835,2.307728,0.176433,0.012777,3284.429889,1.238819,5.874138


In [6]:
df2 = deepcopy(df)
df2.drop('is_cancer', axis=1, inplace=True)
correlation_matrix = df2.corr()
malignancy_corr = correlation_matrix['malignancy'] ################

strong_corr_with_malignancy = malignancy_corr[(malignancy_corr > 0.55) | (malignancy_corr < -0.55)]

strong_corr_with_malignancy = strong_corr_with_malignancy.drop('malignancy') 
print('existem', strong_corr_with_malignancy.shape[0], 'colunas com uma correlacao naquele intervalo')
strong_corr_with_malignancy.head(20)

existem 19 colunas com uma correlacao naquele intervalo


original_shape_LeastAxisLength                     0.597069
original_shape_MajorAxisLength                     0.560898
original_shape_Maximum2DDiameterColumn             0.578477
original_shape_Maximum2DDiameterRow                0.608509
original_shape_Maximum2DDiameterSlice              0.614450
original_shape_Maximum3DDiameter                   0.573497
original_shape_MinorAxisLength                     0.613075
original_shape_SurfaceVolumeRatio                 -0.596664
original_glcm_Id                                   0.560879
original_glcm_Idm                                  0.553955
original_glcm_Imc1                                 0.705151
original_glcm_MCC                                 -0.622857
original_gldm_DependenceNonUniformityNormalized   -0.658191
original_gldm_SmallDependenceEmphasis             -0.606418
original_glrlm_RunLengthNonUniformityNormalized   -0.594241
original_glrlm_RunPercentage                      -0.591054
original_glrlm_ShortRunEmphasis         

In [7]:
strong_corr_columns = strong_corr_with_malignancy.index # nome das colunas que sobraram

subset_df = df[strong_corr_columns] # subset do dataset que tem estas colunas

subset_df = pd.concat([df[['is_cancer']], subset_df], axis=1) # voltar a adicionar (no inicio)

subset_df.head()


,is_cancer,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MinorAxisLength,original_shape_SurfaceVolumeRatio,original_glcm_Id,original_glcm_Idm,original_glcm_Imc1,original_glcm_MCC,original_gldm_DependenceNonUniformityNormalized,original_gldm_SmallDependenceEmphasis,original_glrlm_RunLengthNonUniformityNormalized,original_glrlm_RunPercentage,original_glrlm_ShortRunEmphasis,original_glszm_ZonePercentage,original_ngtdm_Strength
0,True,19.588162,22.827597,31.038346,32.075141,26.977005,34.406945,22.206076,0.368609,0.357261,0.288097,-0.139251,0.533478,0.142832,0.322797,0.814938,0.881115,0.918743,0.363301,1.274217
1,True,21.632902,28.944112,36.659875,30.945300,37.724947,40.384819,23.027779,0.388414,0.453400,0.382826,-0.206279,0.823689,0.105000,0.147483,0.738580,0.842797,0.882628,0.141313,0.926980
2,False,14.561133,26.516200,26.873199,31.855156,25.552264,32.630357,20.380759,0.472904,0.376161,0.295529,-0.138308,0.627493,0.154051,0.255768,0.830722,0.902953,0.927897,0.282061,4.007826
3,True,16.647824,24.114835,26.959597,30.922607,24.750701,30.922607,20.180327,0.361115,0.340451,0.267210,-0.133980,0.612321,0.168861,0.354250,0.829927,0.893385,0.927213,0.405739,1.578325
4,Ambiguous,7.316637,11.451529,11.145619,12.548361,12.934109,13.267970,7.644348,0.816013,0.195261,0.120869,-0.386747,0.707442,0.356263,0.588421,0.942074,0.967286,0.976638,0.662835,5.874138


In [8]:
'''
X = subset_df[strong_corr_columns]  # All 19 columns
y = subset_df['is_cancer']  # Target column

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate Random Forest model
model = RandomForestClassifier(random_state=42)

# Apply RFE to select the top 10 features
rfe = RFE(estimator=model, n_features_to_select=10)  # We want to select 10 out of the 19 features
X_train_rfe = rfe.fit_transform(X_train, y_train)  # Fit RFE and transform the training data
X_test_rfe = rfe.transform(X_test) 

model.fit(X_train_rfe, y_train)

# Predict on the test set
y_pred = model.predict(X_test_rfe)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Get the names of the selected columns, w chat
selected_columns = X.columns[rfe.support_]

print("Selected Features:", selected_columns)
'''

'\nX = subset_df[strong_corr_columns]  # All 19 columns\ny = subset_df[\'is_cancer\']  # Target column\n\n# Train-test split\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# Instantiate Random Forest model\nmodel = RandomForestClassifier(random_state=42)\n\n# Apply RFE to select the top 10 features\nrfe = RFE(estimator=model, n_features_to_select=10)  # We want to select 10 out of the 19 features\nX_train_rfe = rfe.fit_transform(X_train, y_train)  # Fit RFE and transform the training data\nX_test_rfe = rfe.transform(X_test) \n\nmodel.fit(X_train_rfe, y_train)\n\n# Predict on the test set\ny_pred = model.predict(X_test_rfe)\n\naccuracy = accuracy_score(y_test, y_pred)\nprint(f"Accuracy: {accuracy:.4f}")\n\n# Get the names of the selected columns, w chat\nselected_columns = X.columns[rfe.support_]\n\nprint("Selected Features:", selected_columns)\n'

In [9]:
subset_df = pd.read_csv('metadata_3d.csv')

# Step 2: Drop unnecessary columns ('patient_id', 'nodule_idx', and 'malignancy')
subset_df = subset_df.drop(columns=['patient_id', 'nodule_idx', 'malignancy'])

# Step 3: Define the target column 'is_cancer' and the feature set
X = subset_df.drop(columns=['is_cancer'])  # All columns except the target
y = subset_df['is_cancer']  # Target variable

# Step 4: Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Instantiate Random Forest model
model = RandomForestClassifier(random_state=42)

# Step 6: Apply RFE to select the top 10 features
rfe = RFE(estimator=model, n_features_to_select=10)  # Select top 10 features
X_train_rfe = rfe.fit_transform(X_train, y_train)  # Fit RFE and transform training data
X_test_rfe = rfe.transform(X_test)  # Transform test data using the selected features

# Step 7: Fit Random Forest model on the selected features
model.fit(X_train_rfe, y_train)

# Step 8: Predict on the test set
y_pred = model.predict(X_test_rfe)

# Step 9: Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Print accuracy
print(f"Accuracy: {accuracy:.4f}")

# Step 10: Get the names of the selected columns
selected_columns = X.columns[rfe.support_]
print("Selected Features:", selected_columns)

Accuracy: 0.7879
Selected Features: Index(['original_shape_Maximum2DDiameterRow',
       'original_shape_Maximum2DDiameterSlice',
       'original_shape_MinorAxisLength', 'original_glcm_Imc1',
       'original_gldm_DependenceNonUniformity',
       'original_glrlm_RunLengthNonUniformity', 'original_glrlm_RunVariance',
       'original_glszm_GrayLevelNonUniformity', 'original_glszm_ZoneEntropy',
       'original_ngtdm_Strength'],
      dtype='object')


In [10]:













###########################################################################
###################### A PARTIR DAQUI LET ME COOK #########################
###########################################################################

In [11]:
subset_df = pd.read_csv('metadata_3d.csv') # importar outra vez depois organiza se

subset_df = subset_df.drop(columns=['patient_id', 'nodule_idx', 'malignancy'])

X = subset_df.drop(columns=['is_cancer'])  # All columns except the target
y = subset_df['is_cancer']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# y_train.head()

In [46]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [50]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)  # Retains 95% of variance
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Print the number of components selected by PCA
print(f"Number of original components: {X_train.shape[1]}")
print(f"Number of PCA components selected: {X_train_pca.shape[1]}")



Number of original components: 107
Number of PCA components selected: 13


In [44]:
'''
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)
'''

In [45]:
'''
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Initialize the XGBoost model with regularization
xgb_model = XGBClassifier(random_state=42, reg_alpha=0.01, reg_lambda=0.01)

# Step 8: Define hyperparameter search space for tuning
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Step 9: Perform grid search for hyperparameter tuning
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='f1_macro', n_jobs=-1, verbose=2)
grid_search.fit(X_train_pca, y_train) ####################################################################################### fiz isto para tentar ver se e pertinente dar tuning na NN, isto serviu de exemplo

# Use the best estimator found by GridSearchCV, not working as I want but might not be necessary
best_xgb_model = grid_search.best_estimator_

# check performance
y_pred = best_xgb_model.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)

# Train and test using original (scaled) data without PCA to compare
grid_search.fit(X_train_scaled, y_train)

print(f"XGBoost model accuracy with PCA: {accuracy}")

import matplotlib.pyplot as plt
from xgboost import plot_importance

# Assuming grid_search.best_estimator_ is your trained XGBoost model
xgb_model = grid_search.best_estimator_

# Plot feature importance
plt.figure(figsize=(10, 8))
plot_importance(xgb_model, max_num_features=10)  # Adjust max_num_features to show more or fewer
plt.title('Feature Importance')

# Save the plot as a file (PNG in this example)
plt.savefig('xgboost_feature_importance.png', dpi=300, bbox_inches='tight')

plt.close()


Fitting 3 folds for each of 48 candidates, totalling 144 fits
Fitting 3 folds for each of 48 candidates, totalling 144 fits


KeyboardInterrupt: 

<Figure size 1000x800 with 0 Axes>

In [43]:
import pandas as pd
import numpy as np

# Assuming `pca` is your PCA object and `X_train` is your original feature set
pca_components = pca.components_

# Create a DataFrame for better visualization
feature_names = X_train.columns
pca_df = pd.DataFrame(data=pca_components, columns=feature_names)

# Display the contributions of each feature to the PCA components
print(pca_df)


    original_shape_Elongation  original_shape_Flatness  \
0                    0.024207                 0.037132   
1                   -0.000598                 0.026038   
2                   -0.010622                -0.047440   
3                   -0.008471                -0.018809   
4                    0.146337                 0.072297   
5                    0.175349                 0.229114   
6                   -0.220840                -0.290092   
7                    0.202363                 0.121222   
8                   -0.447484                -0.445787   
9                    0.228133                -0.058269   
10                   0.049509                -0.042671   
11                  -0.006186                 0.081519   
12                   0.006089                 0.012731   

    original_shape_LeastAxisLength  original_shape_MajorAxisLength  \
0                         0.151605                        0.144135   
1                        -0.023371             